# Downloading and setting up Julia and Python
Adding all required Packages

In [7]:
!curl -sSL "https://julialang-s3.julialang.org/bin/linux/x64/1.0/julia-1.0.5-linux-x86_64.tar.gz" -o julia.tar.gz
!tar -xzf julia.tar.gz -C /usr --strip-components 1
!rm -rf julia.tar.gz*
!julia -e 'using Pkg; pkg"add IJulia;add OnlineStats; add MAT; precompile"'

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
Precompiling project...


In [8]:
!pip install julia 

In [9]:
import julia
julia.install() 

In [10]:
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main

In [11]:
jl.eval("using OnlineStats,MAT")

In [36]:
Main.eval("""
#Function to generate number of bins (Logarithmically spaced) Given an array length
# I.E. from 2 bins (log(10,2.5)) for the entire length to As many bins that can be fitted in that size of the array
#The case of 1 bin spanning the entire array and Same number of bin as entire length of the array is Manually Added


#https://stackoverflow.com/questions/12418234/logarithmically-spaced-integers
#The values in the array generated by logspace(0, k, n) are the numbers 10^{ik / (n−1)} for 0 ≤ i < n

#Generating Logarithmic spaced numbers 
log_space(st,sp,n,Integ=false,Base=10) = (Integ==true ? (unique(Int64.(floor.(Base .^(range(st, stop=sp, length=n)))))) : (Base .^(range(st, stop=sp, length=n))))

#Generating Logarithmic spaced Integears between  
#log(10,2.5) ----------> 2 Bins
#log(10,endofLine) ----> number of bins nearest to the array-length provided
function GenBinNums(endofLine,num_of_bins=100)
    LogSpacedArray=log_space(log(10,2.5),log(10,endofLine),num_of_bins,true)
    #("Asked for: ",num_of_bins," Digits! Returning: ",length(LogSpacedArray)," Digits!")
    return LogSpacedArray 
end
#GenBinNums(500,10)  #For array of length 500, We asked of 10 bin counts which are logarithmcally spaced

function resrel(signal::Array,time_bins::Int)
    num_of_datapoints=size(signal)[1]
    
    #Creating Balanced Bins
    #See: http://stackoverflow.com/a/21767522
    bins=min(num_of_datapoints,time_bins);
    floor =num_of_datapoints ÷ bins;
    ceiling = floor + 1;
    stepdown = num_of_datapoints % bins;
    lm=ceiling*stepdown;
    
    #Counting Spikes
    ###################################################################
     if(num_of_datapoints % bins==false)
         counts=[sum(elements) for elements in Iterators.partition(signal[:], num_of_datapoints ÷ bins)]
     else
        c1=[sum(elements) for elements in Iterators.partition(signal[1:lm], ceiling)]
        c2=[sum(elements) for elements in Iterators.partition(signal[lm+1:end], floor)]
        counts=vcat(c1,c2)
   ###################################################################     
    end
    #println(sum(counts))
    #return counts
    
    #Eleminating Zero-counts
    zero_loc=findall(counts.==0); #Finding Zero Locations
    deleteat!(counts,zero_loc); #Eleminating Zeros
    
    #Counting unique values and Frequencies
    o=fit!(CountMap(Int), counts); # Couting Unique appearences
    vals=collect(keys(o));freq=collect(values(o)) # Values & their Freqiencies
    
    #for elements in zip(vals,freq); println(elements); end # See
    
    #Checking if total count and Freq/Values matches or not
    sum(counts) == sum(vals.*freq) || throw(AssertionError("Value/Frequency coumputation error!"))
    
    #Calculating Hs & Hk
    M=sum(vals.*freq)
    hs= - sum(  ((freq.*vals)./M) .* log.(M,(vals./M)) )
    hk= - sum(  ((freq.*vals)./M) .* log.(M,((freq.*vals)./M)) )
    
    return [hs,hk]
end

function doit(signal,nbbins)
    result=[resrel(signal,bins) for bins in nbbins]
    return vcat(result'...)
end  
#Sorting and adding theoretical Extremas
function prep(HofS,HofK)
        SHofS=HofS[sortperm(HofS)]
        SHofK=HofK[sortperm(HofS)]

        #Adding Theoretical minimum
        insert!(SHofS,1,0.0);
        insert!(SHofK,1,0.0);

        #Adding Theoretical maximum
        append!(SHofS,1.0);
        append!(SHofK,0.0);
    return SHofS,SHofK
end
function calculate_area(data_points)
    # calculates integral using the trapezoid rule
    return sum(0.5*(abs.(data_points[:,1][1:end-1]+data_points[:,1][2:end]))
        .*(abs.(data_points[:,2][1:end-1]-data_points[:,2][2:end])))
end

# Identifying Neurons

GridN = [7,9,11,13,15,17,19,20,23,24,25,27,28,33,36,37,39,40,41,42,52,60,61,62,63,64,65]
InterN = [8,12,16,22,50]
BorderN= [44]
Others= [ 1,  2,  3,  4,  5,  6, 10, 14, 18, 21, 26, 29, 30, 31, 32, 34, 35,
       38, 43, 45, 46, 47, 48, 49, 51, 53, 54, 55, 56, 57, 58, 59]

function Id(idx::Int64) 
    if idx ∈ GridN  
        return "GridN" 
    elseif idx ∈ InterN 
        return "InterN" 
    elseif idx ∈ Others 
        return "OtherN" 
    elseif idx ∈ BorderN 
        return "BorderN" 
    else 
        return "Error! Idx <=65"
    end
end


function calculate_MSR(binned_spikes)
    nbinsM=GenBinNums(size(binned_spikes)[1],100);
    HSHKm=doit(binned_spikes,nbinsM);
    MySHofS,MySHofK=prep(HSHKm[:,1],HSHKm[:,2]);
    myMSR=calculate_area(hcat(MySHofK,MySHofS));
    return myMSR
end
""")

<PyCall.jlwrap calculate_MSR>

In [18]:
!git clone https://github.com/rcubero/MSR.git

--2021-03-01 16:16:53--  https://raw.githubusercontent.com/rcubero/MSR/master/Flekken_Data/cell_filenames
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1171 (1.1K) [text/plain]
Saving to: ‘cell_filenames’

cell_filenames      100%[===================>]   1.14K  --.-KB/s    in 0s      

2021-03-01 16:16:53 (46.3 MB/s) - ‘cell_filenames’ saved [1171/1171]



In [72]:
import numpy as np
import matplotlib.pyplot as plt

In [73]:

Main.binning_time=10e-3  #10 ms in Secs
Main.highest_time=125293 #in MSecs #No of timedata points

filepath="MSR/Flekken_Data/"

In [78]:
count=1
for filename in open(str(filepath+"cell_filenames"), 'r').readlines():
  Main.file=str.rstrip(filepath+filename)
  Main.eval('data=read(matopen(file), "cellTS")');  # Data Spike time-stamps in Seconds
  Main.eval("edges=0:binning_time:(highest_time*binning_time)");
  binned_spikes=Main.eval("value(fit!(Hist(edges),data))[2]")

  print("Neuron ",count,"(",Main.Id(count),") :",Main.calculate_MSR(binned_spikes))
  count+=1






Neuron  1 ( OtherN ) : 0.28546512360254467
Neuron  2 ( OtherN ) : 0.2912992851190816
Neuron  3 ( OtherN ) : 0.30073136624035784
Neuron  4 ( OtherN ) : 0.29215408811418675
Neuron  5 ( OtherN ) : 0.28047572931896225
Neuron  6 ( OtherN ) : 0.2993795546636262
Neuron  7 ( GridN ) : 0.294675713591849
Neuron  8 ( InterN ) : 0.2621799339187219
Neuron  9 ( GridN ) : 0.2954085108879084
Neuron  10 ( OtherN ) : 0.2672726239200314
Neuron  11 ( GridN ) : 0.27767016026969954
Neuron  12 ( InterN ) : 0.26492524513231813
Neuron  13 ( GridN ) : 0.2919862336354481
Neuron  14 ( OtherN ) : 0.293886963275713
Neuron  15 ( GridN ) : 0.2937983321539725
Neuron  16 ( InterN ) : 0.26542268893184456
Neuron  17 ( GridN ) : 0.2738642076058273
Neuron  18 ( OtherN ) : 0.2805965274169523
Neuron  19 ( GridN ) : 0.2954826796389957
Neuron  20 ( GridN ) : 0.2962065171517621
Neuron  21 ( OtherN ) : 0.297046986004483
Neuron  22 ( InterN ) : 0.2806738660569228
Neuron  23 ( GridN ) : 0.2754394800829103
Neuron  24 ( GridN ) : 0.